# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [14]:
import os
import glob
import psycopg2
import pandas as pd
#from sql_queries import *
import importlib
import sql_queries
importlib.reload(sql_queries)

<module 'sql_queries' from '/home/workspace/sql_queries.py'>

In [15]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [16]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [78]:
song_files = get_files("data/song_data")

In [5]:
song_files[2] 

'/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json'

In [80]:
df = pd.read_json(song_files[4], lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [81]:

song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()
song_data


[['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546]]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [29]:
conn.commit()
cur.execute(sql_queries.song_table_insert, song_data[0])
input = "hemi's name"
cur.execute("select * from songs where title=%s", (input,))
print(cur.fetchone())
conn.commit()

None


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()
artist_data

[['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892]]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
#print(sql_queries.artist_table_insert)
cur.execute(sql_queries.artist_table_insert, artist_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [17]:
log_files = get_files("data/log_data")

In [18]:
df = pd.read_json(log_files[0], lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df.loc[df['page'] == 'NextSong']



In [20]:
t = pd.to_datetime(df['ts'], unit='ms')
#pd.to_datetime(t).dt.weekofyear
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [21]:

time_data = [t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ['start_time', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday']


In [22]:
#List of Series to pandas dataframe, do not understand.. needs to study more
time_df = pd.concat(time_data, axis=1)
time_df.columns = column_labels
time_df.head()


,start_time,hour,day,weekofyear,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [36]:
'''
importlib.reload(sql_queries)
conn.commit()
for i, row in time_df.iterrows():
    cur.execute(sql_queries.time_table_insert, list(row))
    conn.commit()
'''

cur.execute("select to_char(start_time, 'yyyymmdd hh:mi:ss tt') from time;")
print(cur.fetchall())

[('20181130 12:22:07 tt',), ('20181130 01:08:41 tt',), ('20181130 01:12:48 tt',), ('20181130 01:17:05 tt',), ('20181130 01:20:56 tt',), ('20181130 01:24:32 tt',), ('20181130 01:28:19 tt',), ('20181130 01:32:00 tt',), ('20181130 01:38:17 tt',), ('20181130 01:39:53 tt',), ('20181130 01:44:10 tt',), ('20181130 01:48:36 tt',), ('20181130 01:50:28 tt',), ('20181130 01:55:35 tt',), ('20181130 02:01:08 tt',), ('20181130 02:05:24 tt',), ('20181130 02:10:21 tt',), ('20181130 02:15:54 tt',), ('20181130 02:19:19 tt',), ('20181130 03:06:30 tt',), ('20181130 03:10:30 tt',), ('20181130 03:13:03 tt',), ('20181130 03:15:41 tt',), ('20181130 03:21:09 tt',), ('20181130 03:29:23 tt',), ('20181130 03:32:46 tt',), ('20181130 03:37:24 tt',), ('20181130 03:39:31 tt',), ('20181130 03:44:28 tt',), ('20181130 03:49:50 tt',), ('20181130 03:55:42 tt',), ('20181130 03:58:21 tt',), ('20181130 04:00:57 tt',), ('20181130 04:04:13 tt',), ('20181130 04:06:01 tt',), ('20181130 04:09:20 tt',), ('20181130 04:12:58 tt',), 

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [9]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates(['userId'], keep='first')

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [10]:
conn.commit()
for i, row in user_df.iterrows():
    cur.execute(sql_queries.user_table_insert, row)
    conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [13]:
importlib.reload(sql_queries)

# 여기에 문제가 좀 있음 확인이 필요함 
'''
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(sql_queries.song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (songid, row.userId, row.ts, row.level, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(sql_queries.songplay_table_insert, songplay_data)
    conn.commit()
'''
cur.execute("select * from songplays limit 100")
print(cur.fetchall())

[(None, None, '91', Decimal('1543537327796'), 'free', None, '829', 'Dallas-Fort Worth-Arlington, TX', 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)'), (None, None, '73', Decimal('1543540121796'), 'paid', None, '1049', 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"'), (None, None, '73', Decimal('1543540368796'), 'paid', None, '1049', 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"'), (None, None, '73', Decimal('1543540625796'), 'paid', None, '1049', 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"'), (None, None, '73', Decimal('1543540856796'), 'paid', None, '1049', 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; 

In [55]:

for index, row in df.iterrows():
    print(row.song, '+', row.artist, row.length);
    cur.execute(sql_queries.song_select, (row.song, row.artist))
    result = cur.fetchone()
    #print(result)

Jim Henson's Dead + Stephen Lynch 182.85669
Shell Shock + Manowar 247.562
Women Lose Weight (Feat: Slick Rick) + Morcheeba 257.41016
Won't Go Home Without You + Maroon 5 231.23546
Hey_ Soul Sister + Train 216.76363
I'm In Miami Bitch + LMFAO 227.99628
Sexy Bitch + DJ Dizzy 221.1522
The Cure & The Cause (Dennis Ferrer Remix) + Fish Go Deep & Tracey K 377.41669
None + None nan
Staring At Me + M83 96.1824
The Second You Sleep + Saybia 257.25342
Wide Eyes + Local Natives 266.05669
La Resistance (Medley) (LP Version) + South Park 112.97914
Restless + UNKLE Feat. Josh Homme 307.19955
Noc... + Justyna Steczkowska 333.53098
Like You + Evanescence 256.91383
God Put A Smile Upon Your Face + Coldplay 297.35138
Playboy + Hot Chip 333.50485
Every Time You Go + Ellie Goulding 205.06077
Natural Anthem (Album) + Postal Service 307.53914
None + None nan
None + None nan
None + None nan
Taylor + Jack Johnson 240.06485
Naked As We Can + Iron And Wine 153.05098
Fantasy + The xx 158.24934
Epilogue + The Ant

In [23]:
cur.execute('TRUNCATE TABLE artists cascade;')
cur.execute('commit;')

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [37]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.